<a href="https://colab.research.google.com/github/tantowijh/colab-collection/blob/main/MKVExtract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### <img src='https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png' height="45" alt="rclone"/>

In [ ]:
!pip install ipywidgets==7.7.1 &> /dev/null

In [ ]:
# ============================= FORM ============================= #
#@title <font size="4"><b>Step 1. </b>Upload your <code>rclone.conf</code> profile</font> { vertical-output: true }
#@markdown <h6>*<i>Required to mount your drive!</i></h4>
## @markdown <h7>Your <code>rclone.conf</code> file is located at:</i></h4>
# @markdown ### <font size=1px><i>For not able to upload local file : https://stackoverflow.com/a/58661947</font>
# ================================================================ #
MODE = "RCONFIG"
from os import path as _p
from IPython.display import clear_output

if not _p.exists("/root/.ipython/rlab_utils.py"):
    from shlex import split as _spl
    from subprocess import run  # nosec

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/RLabClone/master/res/rlab_utils.py \
                    -O /root/.ipython/rlab_utils.py"
    run(_spl(shellCmd))  # nosec

import importlib, rlab_utils
from google.colab import files  # pylint: disable=import-error #nosec
from rlab_utils import checkAvailable, runSh, PATH_RClone_Config, prepareSession


def generateUploadList():
    prepareSession()
    if checkAvailable("/content/upload.txt"):
        runSh("rm -f upload.txt")
    runSh(
        f"rclone --config {PATH_RClone_Config}/rclone.conf lsf {REMOTE}: --include '{QUERY_PATTERN}' --drive-shared-with-me --files-only --max-depth 1 > /content/upload.txt",
        shell=True,  # nosec
    )


def uploadLocalFiles():
    prepareSession()
    if MODE == "UTILS":
        filePath = "/root/.ipython/rlab_utils.py"
    elif MODE in ("RCONFIG", "RCONFIG_append"):
        filePath = f"{PATH_RClone_Config}/rclone.conf"
    else:
        pass

    try:
      if checkAvailable(filePath):
        runSh(f"rm -f {filePath}")
      print("Select file from your computer.\n")
      uploadedFile = files.upload()
      fileNameDictKeys = uploadedFile.keys()
      fileNo = len(fileNameDictKeys)
      if fileNo > 1:
          for fn in fileNameDictKeys:
              runSh(f'rm -f "/content/{fn}"')
          return print("\nPlease only upload a single config file.")
      elif fileNo == 0:
          return print("\nFile upload cancelled.")
      elif fileNo == 1:
          for fn in fileNameDictKeys:
              if checkAvailable(f"/content/{fn}"):
                  if MODE == "RCONFIG_append":
                    import urllib
                    urllib.request.urlretrieve("https://biplobsd.github.io/RLabClone/res/rclonelab/rclone.conf",
                                               "/usr/local/sessionSettings/rclone.conf")
                    with open(f"/content/{fn}", 'r+') as r:
                      new_data = r.read()
                      runSh(f'rm -f "/content/{fn}"')
                    with open(filePath, 'r+') as f:
                      old_data = f.read()
                      f.seek(0)
                      f.truncate(0)
                      f.write(old_data + new_data)
                    print("\nUpdate completed.")
                  else:
                    runSh(f'mv -f "/content/{fn}" {filePath}')
                    runSh(f"chmod 666 {filePath}")
                    runSh(f'rm -f "/content/{fn}"')
                    importlib.reload(rlab_utils)
                    print("Upload completed.")
              return
      else:
          print("\nNo file")
          return
    except:
      return print("\nUpload process Error.")


if MODE == "GENERATELIST":
    generateUploadList()
else:
    uploadLocalFiles()


Select file from your computer.



Saving rclone.conf to rclone.conf
Upload completed.


In [ ]:
# ============================= FORM ============================= #
#@title <font size="4"><b>Step 2</b>. Mount your Drive with rclone</font> { vertical-output: true }


Cache_Directory = "DISK"
import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re
#####################################
if not os.path.exists("/root/.ipython/rlab_utils.py"):
  from shlex import split as _spl
  from subprocess import run

  shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                  -O /root/.ipython/rlab_utils.py"
  run(_spl(shellCmd))

from rlab_utils import (
    runSh,
    prepareSession,
    PATH_RClone_Config,
)

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#127ACC;margin-top:0px;">
                  Choose Drive to Mount:</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))

prepareSession()
content = open(f"{PATH_RClone_Config}/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)

if Cache_Directory == 'RAM':
  cache_path = '/dev/shm'
elif Cache_Directory == 'DISK':
  os.makedirs('/tmp', exist_ok=True)
  cache_path = '/tmp'

def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    cmd = rf"rclone mount {mountNam.value}: {mPoint}" \
      rf" --config {PATH_RClone_Config}/rclone.conf" \
      ' --daemon'

    if runSh(cmd, shell=True) == 0:
      print(f"Mount success! - \t{mPoint}")
    else:
      print(f"Mount failed! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"Unmounted success! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)

ShowAC()

Mount success! - 	/content/drives/MyMega


### ⬅️ **Mount Google Drive**
Run the following command to mount google drive:

In [ ]:
#@title **[0] Link your Google Drive**
#@markdown <center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#@title **[1] Run this code if you want a longer run time (Optional but recommended)</font>**
#@markdown *This cell runs a JS code that automatically reconnects you to avoid inactivity due to idle.*

import IPython
from IPython.display import clear_output
from google.colab import output

display(IPython.display.Javascript('''
 function connectRefresher() {
       window.ConnectButtonIntervalId = setInterval(function ConnectButton(){
                console.log("connected"); 
                document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
                document.querySelector("colab-sessions-dialog").shadowRoot.querySelector("#footer > div > paper-button").click();
                console.log("closed the dialog!!"); 
            },60000);
    }
    
 function clearRefresher() { 
           console.log("clear Interval called !!");
           clearInterval(window.ConnectButtonIntervalId);
    }

 connectRefresher(); //to connect the refresher
 clearRefresher(); //to disconnect the refresher
'''))

clear_output()

### ⬅️ **MKV Extractor**

In [ ]:
#@markdown <h3>➡️ Click Here to Install FFmpeg, VCSI & Mkvtoolnix</h3>

from IPython.display import clear_output
import os, sys, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/yunooooo/gcct/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing ffmpeg...", ty='twg')
!apt install ffmpeg &> /dev/null
textAn("Installing mkvtoolnix...", ty='twg')
!sudo apt-get install mkvtoolnix &> /dev/null
textAn("Installing vcsi...", ty='twg') 
!pip install vcsi &> /dev/null


from IPython.display import HTML, clear_output
clear_output()
print("Server Started Successfully")

Server Started Successfully


##### ⬅️ **MKVToolnix Mode**

In [ ]:
#@title <h3>➡️ Generate MKV Information</h3>
FilePath = "" #@param {type:"string"}
!mkvmerge -i '{FilePath}'

In [ ]:
#@title <h3>➡️ Start MKV Extract</h3>

#Input and output file
FilePath = "" #@param {type:"string"}
SameFolderExtract = True #@param {type:"boolean"}
#@markdown >You can leave output path empty if extract in the same folder is checked
OutputPath = "" #@param {type:"string"}
from pathlib import Path
Path(OutputPath).mkdir(parents=True, exist_ok=True)
OutputType = input("Please enter export file type... \nExample (srt, ass, vtt, etc) \nFile type: ")
Type = '.' + OutputType

#Track Initiation
StartTrack = None #@param {type:"integer"}
StartTrack = StartTrack-1
EndTrack = None #@param {type:"integer"}
EndTrack = EndTrack-1

#Renaming file
Fix = len(".mkv")
src =f"{FilePath}"
dst =f"{src[:-(int(Fix))]}"
if SameFolderExtract == True:
  dst =f"{dst}_track"
else:
  sparator = '/'
  for i in range(1,11):
    # Remove all characters before the character '-' from string
    before, sparator, after = dst.partition('/')
    if len(after) > 0:
      dst = after
  dst =f"{OutputPath}/{dst}"

lang = input("Please input track specs... \nExample (audio, video, subtitle, id, eng, ms, jp, etc) \n Specs: ")

# original command = mkvextract tracks myFile.mkv 3:myFile1.srt 4:myFile2.srt

for Tracks in range(EndTrack):
  StartTrack = StartTrack+1
  Track = StartTrack
  !mkvextract tracks '{FilePath}' '{Track}:{dst}{Track}.{lang}{Type}'

##### ⬅️ **FFmpeg Mode**

In [ ]:
#@title <h3>➡️ Generate MKV Information</h3>
FilePath = "/content/War_God_System!_-_05_(B-Global_Donghua_1920x1080_HEVC_AAC_MKV).mkv" #@param {type:"string"}
!ffprobe '{FilePath}' 2>&1 >/dev/null | grep Stream

In [ ]:
#@title <h3>➡️ Start MKV Extract</h3>

from IPython.display import clear_output

#Input and output file
FilePath = "/content/drive/Shareddrives/06. YT-DLP/[NC-Raws] \u5373\u4F7F\u5982\u6B64\u4F9D\u65E7\u6B65\u6B65\u7D27\u903C - 07 (B-Global 1920x1080 HEVC AAC MKV) [5FA479B2].mkv" #@param {type:"string"}
SameFolderExtract = False #@param {type:"boolean"}
#@markdown >You can leave output path empty if extract in the same folder is checked
OutputPath = "/content" #@param {type:"string"}
from pathlib import Path
Path(OutputPath).mkdir(parents=True, exist_ok=True)
OutputType = input("Please enter export file type... \nExample (srt, ass, vtt, etc) \nFile type: ")
Type = '.' + OutputType

#Track Initiation
StartTrack = 0 #@param {type:"integer"}
StartTrack = StartTrack-1
EndTrack = 1 #@param {type:"integer"}
EndTrack = EndTrack

#Renaming file
Fix = len(".mkv")
src =f"{FilePath}"
dst =f"{src[:-(int(Fix))]}"
if SameFolderExtract == True:
  dst =f"{dst}_track"
else:
  sparator = '/'
  for i in range(1,11):
    # Remove all characters before the character '-' from string
    before, sparator, after = dst.partition('/')
    if len(after) > 0:
      dst = after
  dst =f"{OutputPath}/{dst}_track"
!ffprobe '{FilePath}' 2>&1 >/dev/null | grep Stream > namingspecslist.txt
import re
for Tracks in range(StartTrack,EndTrack):
  StartTrack = StartTrack+1
  Track = StartTrack
  x = Track+1
  fname = open("namingspecslist.txt")
  cuk = !sed -n {x}p namingspecslist.txt
  specs = str(cuk)
  pattern = '\(.*?\)'
  findid = re.search(pattern, specs)
  specsresult = findid.group()
  scr =f"{specsresult}"
  id = scr.replace("(", "")
  id = id.replace(")", "")
  !ffmpeg -y -v error -stats -txt_format text -i '{FilePath}' -map 0:{Track} '{dst}{Track}.{id}{Type}'

### ⬅️ **File Rename**

In [ ]:
#@title <h3>➡️ Renaming File</h3>
import os
import signal
import sys

def sigint_handler(signal, frame):
    print ('KeyboardInterrupt is caught')
    sys.exit(0)
signal.signal(signal.SIGINT, sigint_handler)

InputLocation = "" #@param {type:"string"}

Text2Replace  = " " #@param {type:"string"}
ReplacementText   = "_" #@param {type:"string"}

FileType = "srt" #@param ["Write it yourself...", "mp4", "mkv", "png", "jpg", "jpeg", "ass", "srt"]
if FileType == "Write it yourself...":
  ftype = input('Enter the file type to rename: ') 
  Extension = '.' + ftype
else:
  Extension = '.' + FileType

PutLabel = False #@param {type:"boolean"}
Label = "" #@param {type:"string"} 

fix = len(Extension)
#-(int(fix))

# Renaming the file

def RenameMe():
  for count, filename in enumerate(os.listdir(InputLocation)):
    if(filename.endswith(Extension)):
          src =f"{InputLocation}/{filename}"  # foldername/filename, if .py file is outside folder
          cuk =f"{filename}"
          fname = cuk.replace(Text2Replace, ReplacementText)
          dst =f"{InputLocation}/{fname}"
          
          # rename() function will
          # rename all the files with the pre defined extension
          os.rename(src, dst)


def LabelMe():
  for count, filename in enumerate(os.listdir(InputLocation)):
    if(filename.endswith(Extension)):
          src =f"{InputLocation}/{filename}"  # foldername/filename, if .py file is outside folder
          dst =f"{InputLocation}/{Label} {filename}"
          
          # rename() function will
          # rename all the files with the pre defined extension
          os.rename(src, dst)

if PutLabel == False:
  RenameMe()
else:
  LabelMe()